In [31]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '4' 

import deepchem as dc
import pandas as pd
import pickle
import numpy as np
import warnings

from utils import *
warnings.filterwarnings('ignore')

freesolv = pd.read_csv('dicts/database.txt',sep='; ')
# freesolv['uncertainty'] = ~freesolv['text notes.'].str.contains('Experimental uncertainty not presently available, so assigned a default value')#.astype(int)+2
freesolv['uncertainty'] = ~freesolv['text notes.'].str.contains('default')
# freesolv['uncertainty'] = ~freesolv['text notes.'].str.contains('uncertainty')
freesolv#.describe()

,compound id (and file prefix),SMILES,iupac name (or alternative if IUPAC is unavailable or not parseable by OEChem),experimental value (kcal/mol),experimental uncertainty (kcal/mol),Mobley group calculated value (GAFF) (kcal/mol),calculated uncertainty (kcal/mol),experimental reference (original or paper this value was taken from),calculated reference,text notes.,uncertainty
0,mobley_1017962,CCCCCC(=O)OC,methyl hexanoate,-2.49,0.6,-3.30,0.03,10.1021/ct050097l,10.1021/acs.jced.7b00104,Experimental uncertainty not presently availab...,False
1,mobley_1019269,CCCCO,butan-1-ol,-4.72,0.6,-3.23,0.03,10.1021/ct050097l,10.1021/acs.jced.7b00104,Experimental uncertainty not presently availab...,False
2,mobley_1034539,c1cc(c(cc1c2cc(c(c(c2Cl)Cl)Cl)Cl)Cl)Cl,"1,2,3,4-tetrachloro-5-(3,4-dichlorophenyl)benzene",-3.04,0.1,-1.08,0.04,10.1007/s10822-012-9568-8,10.1021/acs.jced.7b00104,Experimental uncertainty not presently availab...,False
3,mobley_1036761,C1CCC(CC1)N,cyclohexanamine,-4.59,0.6,-3.95,0.03,10.1021/ct050097l,10.1021/acs.jced.7b00104,Experimental uncertainty not presently availab...,False
4,mobley_1046331,c1ccc(cc1)OC=O,phenyl formate,-3.82,0.6,-5.44,0.03,"J. Peter Guthrie, unpublished data, as provide...",10.1021/acs.jced.7b00104,Experimental uncertainty not presently availab...,False
...,...,...,...,...,...,...,...,...,...,...,...
637,mobley_9913368,C(=C/Cl)\Cl,"(E)-1,2-dichloroethylene",-0.78,0.6,1.02,0.02,10.1021/ct050097l,10.1021/acs.jced.7b00104,"Renamed mobley_9913368 from (E)-1,2-dichloroet...",True
638,mobley_9942801,CCc1ccc(cc1)C,1-ethyl-4-methyl-benzene,-0.95,0.6,-0.57,0.03,10.1021/ct050097l,10.1021/acs.jced.7b00104,Experimental uncertainty not presently availab...,False
639,mobley_994483,CCBr,bromoethane,-0.74,0.6,0.49,0.02,10.1021/ct050097l,10.1021/acs.jced.7b00104,Experimental uncertainty not presently availab...,False
640,mobley_9974966,CC(C)SC(C)C,2-isopropylsulfanylpropane,-1.21,0.6,0.14,0.03,10.1021/ct050097l,10.1021/acs.jced.7b00104,Experimental uncertainty not presently availab...,False


In [39]:
# freesolv[(freesolv.uncertainty == 1)]['experimental uncertainty (kcal/mol)'].value_counts()
freesolv[(freesolv.uncertainty == 1)].describe()

,experimental value (kcal/mol),experimental uncertainty (kcal/mol),Mobley group calculated value (GAFF) (kcal/mol),calculated uncertainty (kcal/mol)
count,59.000000,59.000000,59.000000,59.000000
mean,-2.760169,0.478305,-2.697458,0.029492
std,3.770399,0.414463,4.986616,0.010574
min,-11.000000,0.100000,-17.240000,0.010000
25%,-4.565000,0.200000,-3.710000,0.020000
50%,-2.700000,0.600000,-1.980000,0.030000
75%,-0.050000,0.600000,0.975000,0.030000
max,2.970000,1.930000,3.030000,0.070000


In [49]:
# freesolv[(freesolv.uncertainty == 0)]['experimental uncertainty (kcal/mol)'].value_counts()
freesolv[(freesolv.uncertainty == 0)].describe()

,experimental value (kcal/mol),experimental uncertainty (kcal/mol),Mobley group calculated value (GAFF) (kcal/mol),calculated uncertainty (kcal/mol)
count,583.000000,583.000000,583.000000,583.000000
mean,-3.908542,0.576364,-3.565866,0.030412
std,3.842985,0.291562,4.093238,0.008861
min,-25.470000,0.030000,-21.760000,0.010000
25%,-5.730000,0.600000,-5.445000,0.030000
50%,-3.640000,0.600000,-3.130000,0.030000
75%,-1.385000,0.600000,-0.605000,0.030000
max,3.430000,1.930000,3.330000,0.090000


In [38]:
freesolv[(freesolv['experimental uncertainty (kcal/mol)'] >= 1.0)&(freesolv.uncertainty == 0)].describe()

,experimental value (kcal/mol),experimental uncertainty (kcal/mol),Mobley group calculated value (GAFF) (kcal/mol),calculated uncertainty (kcal/mol)
count,31.000000,31.000000,31.000000,31.000000
mean,-7.130645,1.493871,-7.071290,0.040968
std,4.163224,0.418479,5.223474,0.006509
min,-16.430000,1.000000,-16.040000,0.030000
25%,-9.690000,1.000000,-11.530000,0.040000
50%,-5.660000,1.370000,-5.380000,0.040000
75%,-3.675000,1.930000,-2.425000,0.040000
max,-2.330000,1.930000,0.760000,0.060000


In [47]:
freesolv[(freesolv['experimental value (kcal/mol)'] < -11.98)].describe()

,experimental value (kcal/mol),experimental uncertainty (kcal/mol),Mobley group calculated value (GAFF) (kcal/mol),calculated uncertainty (kcal/mol)
count,15.000000,15.000000,15.000000,15.000000
mean,-17.330667,0.800667,-16.232000,0.044000
std,3.723210,0.566751,2.699397,0.019928
min,-25.470000,0.160000,-21.760000,0.030000
25%,-18.445000,0.295000,-17.680000,0.030000
50%,-16.920000,0.740000,-16.610000,0.040000
75%,-14.835000,1.050000,-14.640000,0.045000
max,-12.640000,1.930000,-10.140000,0.090000


In [52]:
amino_analogues = ['methane','propane','isobutane','n-butane','methanol','ethanol','methanethiol','methyl ethyl sulfide',
                   'acetamide','propionamide','toluene','p-cresol','1-methylimidazole','methylindole']
freesolv[(freesolv['iupac name (or alternative if IUPAC is unavailable or not parseable by OEChem)'].isin(amino_analogues))]
#not found: methyl ethyl sulfide, methylindole

,compound id (and file prefix),SMILES,iupac name (or alternative if IUPAC is unavailable or not parseable by OEChem),experimental value (kcal/mol),experimental uncertainty (kcal/mol),Mobley group calculated value (GAFF) (kcal/mol),calculated uncertainty (kcal/mol),experimental reference (original or paper this value was taken from),calculated reference,text notes.,uncertainty
37,mobley_1636752,CO,methanol,-5.10,0.6,-3.49,0.02,10.1021/ct050097l,10.1021/acs.jced.7b00104,Experimental uncertainty not presently availab...,False
66,mobley_1873346,Cc1ccccc1,toluene,-0.90,0.2,-0.79,0.03,10.1039/P29900000291,10.1021/acs.jced.7b00104,Experimental uncertainty as suggested by 10.10...,True
76,mobley_1923244,CCCC,n-butane,2.10,0.2,2.59,0.02,10.1039/P29900000291,10.1021/acs.jced.7b00104,Experimental uncertainty as suggested by 10.10...,True
93,mobley_2068538,CCC,propane,2.00,0.2,2.50,0.02,10.1039/P29900000291,10.1021/acs.jced.7b00104,Experimental uncertainty as suggested by 10.10...,True
112,mobley_2310185,CCO,ethanol,-5.00,0.6,-3.39,0.02,10.1021/ct050097l,10.1021/acs.jced.7b00104,Experimental uncertainty not presently availab...,False
168,mobley_2925352,Cc1ccc(cc1)O,p-cresol,-6.13,0.6,-5.58,0.03,10.1021/ct050097l,10.1021/acs.jced.7b00104,Experimental uncertainty not presently availab...,False
326,mobley_5157661,CC(C)C,isobutane,2.30,0.2,2.54,0.02,10.1039/P29900000291,10.1021/acs.jced.7b00104,Experimental uncertainty as suggested by 10.10...,True
329,mobley_525934,CS,methanethiol,-1.20,0.2,-0.27,0.02,10.1039/P29900000291,10.1021/acs.jced.7b00104,Experimental uncertainty as suggested by 10.10...,True
349,mobley_5499659,Cn1ccnc1,1-methylimidazole,-8.41,0.6,-6.28,0.02,10.1021/ct050097l,10.1021/acs.jced.7b00104,Experimental uncertainty not presently availab...,False
524,mobley_8048190,CC(=O)N,acetamide,-9.71,0.6,-8.82,0.02,10.1021/ct050097l,10.1021/acs.jced.7b00104,Experimental uncertainty not presently availab...,False
